# Import

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
import os
from PIL import Image, ImageDraw
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle

import os
import glob
from tqdm import tqdm

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler, StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, confusion_matrix

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchvision.models import resnet50
# import tensorflow as tf
# from tensorflow.keras import layers, Model
# import tensorflow.keras.backend as K
# from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
# from tensorflow.keras.models import Model, load_model, Sequential
# from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, Input, Conv2D, Add, Layer, Multiply, Concatenate
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.applications import ResNet50, EfficientNetB0
# from tensorflow.keras.applications.efficientnet import preprocess_input
# from tensorflow.keras.applications.resnet50 import preprocess_input
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# from tensorflow.keras.initializers import HeNormal
# from tensorflow.keras.losses import Loss
# from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Machine Learning

In [177]:
co_1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA/Coordinate.csv')
co_2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_2/Coordinate.csv')
co_3 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

co_1.drop('Unnamed: 0', axis=1, inplace=True)
co_2.drop('Unnamed: 0', axis=1, inplace=True)
co_3.drop('Unnamed: 0', axis=1, inplace=True)

co = pd.concat([co_1,co_2, co_3], join='outer', axis=0, ignore_index=True)
co = co - 13

co.reset_index(inplace=True)
co.rename(columns = {'index': 'image_name'}, inplace = True)

co['image_name'] = co['image_name'] + 1



df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA/Data.xlsx')
df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_2/Data.xlsx')
df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')

df_1.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '크라운해태 LPBA 챔피언십 2024 한가위'], axis=1, inplace=True)
df_2.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '2024 LPBA SYBAZZAR Hanoi Open'], axis=1, inplace=True)
df_3.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '휴온스 LPBA 챔피언십'], axis=1, inplace=True)

df = pd.concat([df_1, df_2, df_3], join='outer', axis=0, ignore_index=True)
df['image_name'] = co['image_name']



coordinate_columns = ['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']
co_clean = co.dropna(subset=coordinate_columns)

merged_df = pd.merge(co_clean, df, on='image_name')
merged_df.drop('image_name', axis=1, inplace=True)

In [178]:
# 공 간의 거리 계산 함수 (소수점 둘째자리 반올림 포함)
def calculate_distance(x1, y1, x2, y2):
    return np.round(np.sqrt((x2 - x1)**2 + (y2 - y1)**2), 2)

# 공 간의 각도 계산 함수 (소수점 둘째자리 반올림 포함)
def calculate_angle(x1, y1, x2, y2):
    return np.round(np.degrees(np.arctan2(y2 - y1, x2 - x1)), 2)

# 이미지의 가장자리로부터 거리 계산 함수 (소수점 둘째자리 반올림 포함)
def distance_to_edge(x, y, img_width, img_height):
    return {
        'left_edge': np.round(x, 2),
        'right_edge': np.round(img_width - x, 2),
        'top_edge': np.round(y, 2),
        'bottom_edge': np.round(img_height - y, 2)
    }



# 공 간의 거리 추가
merged_df['distance_cue_to_1'] = calculate_distance(
    merged_df['큐볼_x'], merged_df['큐볼_y'], merged_df['적구_1_x'], merged_df['적구_1_y']
)

merged_df['distance_cue_to_2'] = calculate_distance(
    merged_df['큐볼_x'], merged_df['큐볼_y'], merged_df['적구_2_x'], merged_df['적구_2_y']
)

merged_df['distance_1_to_2'] = calculate_distance(
    merged_df['적구_1_x'], merged_df['적구_1_y'], merged_df['적구_2_x'], merged_df['적구_2_y']
)



# 공 간의 각도 추가
merged_df['angle_cue_to_1'] = calculate_angle(
    merged_df['큐볼_x'], merged_df['큐볼_y'], merged_df['적구_1_x'], merged_df['적구_1_y']
)

merged_df['angle_cue_to_2'] = calculate_angle(
    merged_df['큐볼_x'], merged_df['큐볼_y'], merged_df['적구_2_x'], merged_df['적구_2_y']
)

merged_df['angle_1_to_2'] = calculate_angle(
    merged_df['적구_1_x'], merged_df['적구_1_y'], merged_df['적구_2_x'], merged_df['적구_2_y']
)



# 이미지 가장자리로부터의 절대적 거리 추가
img_width, img_height = 753, 376

edges_cue = merged_df.apply(
    lambda row: distance_to_edge(row['큐볼_x'], row['큐볼_y'], img_width, img_height), axis=1
)
edges_1 = merged_df.apply(
    lambda row: distance_to_edge(row['적구_1_x'], row['적구_1_y'], img_width, img_height), axis=1
)
edges_2 = merged_df.apply(
    lambda row: distance_to_edge(row['적구_2_x'], row['적구_2_y'], img_width, img_height), axis=1
)



# 가장자리 거리 정보 병합
merged_df['cue_left_edge'] = edges_cue.apply(lambda x: x['left_edge'])
merged_df['cue_right_edge'] = edges_cue.apply(lambda x: x['right_edge'])
merged_df['cue_top_edge'] = edges_cue.apply(lambda x: x['top_edge'])
merged_df['cue_bottom_edge'] = edges_cue.apply(lambda x: x['bottom_edge'])

merged_df['1_left_edge'] = edges_1.apply(lambda x: x['left_edge'])
merged_df['1_right_edge'] = edges_1.apply(lambda x: x['right_edge'])
merged_df['1_top_edge'] = edges_1.apply(lambda x: x['top_edge'])
merged_df['1_bottom_edge'] = edges_1.apply(lambda x: x['bottom_edge'])

merged_df['2_left_edge'] = edges_2.apply(lambda x: x['left_edge'])
merged_df['2_right_edge'] = edges_2.apply(lambda x: x['right_edge'])
merged_df['2_top_edge'] = edges_2.apply(lambda x: x['top_edge'])
merged_df['2_bottom_edge'] = edges_2.apply(lambda x: x['bottom_edge'])

In [179]:
merged_df

,큐볼_x,큐볼_y,적구_1_x,적구_1_y,적구_2_x,적구_2_y,Results,distance_cue_to_1,distance_cue_to_2,distance_1_to_2,...,cue_top_edge,cue_bottom_edge,1_left_edge,1_right_edge,1_top_edge,1_bottom_edge,2_left_edge,2_right_edge,2_top_edge,2_bottom_edge
0,565.0,186.0,378.0,184.0,191.0,137.0,0,187.01,377.20,192.82,...,186.0,190.0,378.0,375.0,184.0,192.0,191.0,562.0,137.0,239.0
1,357.0,46.0,84.0,277.0,197.0,320.0,0,357.62,317.29,120.90,...,46.0,330.0,84.0,669.0,277.0,99.0,197.0,556.0,320.0,56.0
2,533.0,356.0,553.0,200.0,689.0,242.0,0,157.28,193.21,142.34,...,356.0,20.0,553.0,200.0,200.0,176.0,689.0,64.0,242.0,134.0
3,21.0,21.0,563.0,112.0,711.0,131.0,0,549.59,698.71,149.21,...,21.0,355.0,563.0,190.0,112.0,264.0,711.0,42.0,131.0,245.0
4,496.0,15.0,704.0,298.0,712.0,128.0,1,351.22,243.77,170.19,...,15.0,361.0,704.0,49.0,298.0,78.0,712.0,41.0,128.0,248.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,220.0,250.0,22.0,308.0,110.0,345.0,1,206.32,145.34,95.46,...,250.0,126.0,22.0,731.0,308.0,68.0,110.0,643.0,345.0,31.0
6264,116.0,345.0,33.0,219.0,151.0,349.0,0,150.88,35.23,175.57,...,345.0,31.0,33.0,720.0,219.0,157.0,151.0,602.0,349.0,27.0
6265,617.0,345.0,454.0,54.0,256.0,279.0,1,333.54,366.98,299.71,...,345.0,31.0,454.0,299.0,54.0,322.0,256.0,497.0,279.0,97.0
6266,469.0,337.0,456.0,81.0,446.0,236.0,0,256.33,103.59,155.32,...,337.0,39.0,456.0,297.0,81.0,295.0,446.0,307.0,236.0,140.0


In [66]:
# Visual Python: Machine Learning > Pipeline
# [1] Data Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(merged_df[['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y', 'distance_cue_to_1', 'distance_cue_to_2', 'distance_1_to_2', 'angle_cue_to_1', 'angle_cue_to_2', 'angle_1_to_2', 'cue_left_edge', 'cue_right_edge', 'cue_top_edge', 'cue_bottom_edge', '1_left_edge', '1_right_edge', '1_top_edge', '1_bottom_edge', '2_left_edge', '2_right_edge', '2_top_edge', '2_bottom_edge']], merged_df['Results'], random_state=123)

# [2] Classifier
from xgboost import XGBClassifier

model = XGBClassifier()

# [3] Fit
model.fit(X_train, y_train)

# [4] Predict
pred = model.predict(X_test)

# [5] Evaluation
from sklearn import metrics
from IPython.display import display, Markdown
# Confusion Matrix
display(Markdown('### Confusion Matrix'))
display(pd.crosstab(y_test, pred, margins=True))
# Classification report
print(metrics.classification_report(y_test, pred))

### Confusion Matrix

col_0,0,1,All
Results,,,
0,579,306,885
1,374,308,682
All,953,614,1567


              precision    recall  f1-score   support

           0       0.61      0.65      0.63       885
           1       0.50      0.45      0.48       682

    accuracy                           0.57      1567
   macro avg       0.55      0.55      0.55      1567
weighted avg       0.56      0.57      0.56      1567



In [67]:
# [4] Predict
pred_2 = model.predict(X_train)

# [5] Evaluation
from sklearn import metrics
from IPython.display import display, Markdown
# Confusion Matrix
display(Markdown('### Confusion Matrix'))
display(pd.crosstab(y_train, pred_2, margins=True))
# Classification report
print(metrics.classification_report(y_train, pred_2))

### Confusion Matrix

col_0,0,1,All
Results,,,
0,2545,46,2591
1,50,2060,2110
All,2595,2106,4701


              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2591
           1       0.98      0.98      0.98      2110

    accuracy                           0.98      4701
   macro avg       0.98      0.98      0.98      4701
weighted avg       0.98      0.98      0.98      4701



---

In [180]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 'Results' 열을 제외한 나머지 특성들만 선택
X = merged_df.drop(columns=['Results'])

# 데이터 표준화: PCA 전에 데이터를 표준화하는 것이 중요합니다.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA 모델 생성 (6개의 주성분으로 축소)
pca = PCA(n_components=6)

# PCA 변환 수행
X_pca = pca.fit_transform(X_scaled)

# 변환된 데이터프레임을 pandas DataFrame으로 변환하여 열 이름 추가
X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(6)])

# 결과 출력: PCA 변환된 데이터
print(X_pca_df.head())

# 만약 원본 데이터프레임에 PCA 결과를 추가하고 싶다면:
merged_df_pca = pd.concat([merged_df['Results'], X_pca_df], axis=1)

        PC1       PC2       PC3       PC4       PC5       PC6
0  0.152950 -0.612932  1.267224  1.064086 -1.289801 -1.469757
1 -2.326904 -0.496510 -2.723680 -2.426623  0.391026 -1.478862
2  2.146738  1.979593  2.373301 -0.714590  0.885107  0.525698
3  0.979152 -1.476844 -2.225612  1.222697  1.410778  3.393772
4  3.080788 -2.960326 -0.850021 -1.672188  0.368715  1.342354


In [181]:
merged_df_pca

,Results,PC1,PC2,PC3,PC4,PC5,PC6
0,0,0.152950,-0.612932,1.267224,1.064086,-1.289801,-1.469757
1,0,-2.326904,-0.496510,-2.723680,-2.426623,0.391026,-1.478862
2,0,2.146738,1.979593,2.373301,-0.714590,0.885107,0.525698
3,0,0.979152,-1.476844,-2.225612,1.222697,1.410778,3.393772
4,1,3.080788,-2.960326,-0.850021,-1.672188,0.368715,1.342354
...,...,...,...,...,...,...,...
6263,1,-3.954858,0.982267,-0.442768,-2.681865,0.335528,-1.139918
6264,0,-4.019874,3.000613,0.946111,-0.597139,0.584653,-0.408236
6265,1,0.904581,3.630116,1.317308,1.196700,-1.299754,-1.146109
6266,0,0.996410,2.747549,1.357340,1.175204,-0.101353,-0.012071


In [228]:
# Visual Python: Machine Learning > Pipeline
# [1] Data Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(merged_df_pca[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']], merged_df_pca['Results'])

# [2] Classifier
from xgboost import XGBClassifier

model = XGBClassifier()

# [3] Fit
model.fit(X_train, y_train)

# [4] Predict
pred = model.predict(X_test)

# [5] Evaluation
from sklearn import metrics
from IPython.display import display, Markdown
# Confusion Matrix
display(Markdown('### Confusion Matrix'))
display(pd.crosstab(y_test, pred, margins=True))
# Classification report
print(metrics.classification_report(y_test, pred))

### Confusion Matrix

col_0,0,1,All
Results,,,
0,518,344,862
1,381,324,705
All,899,668,1567


              precision    recall  f1-score   support

           0       0.58      0.60      0.59       862
           1       0.49      0.46      0.47       705

    accuracy                           0.54      1567
   macro avg       0.53      0.53      0.53      1567
weighted avg       0.54      0.54      0.54      1567



# The New CNN

## ResNet

In [ ]:
co_1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA/Coordinate.csv')
co_2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_2/Coordinate.csv')
co_3 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

co_1.drop('Unnamed: 0', axis=1, inplace=True)
co_2.drop('Unnamed: 0', axis=1, inplace=True)
co_3.drop('Unnamed: 0', axis=1, inplace=True)

co = pd.concat([co_1,co_2, co_3], join='outer', axis=0, ignore_index=True)
co = co - 13

co.reset_index(inplace=True)
co.rename(columns = {'index': 'image_name'}, inplace = True)

co['image_name'] = co['image_name'] + 1



df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA/Data.xlsx')
df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_2/Data.xlsx')
df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')

df_1.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '크라운해태 LPBA 챔피언십 2024 한가위'], axis=1, inplace=True)
df_2.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '2024 LPBA SYBAZZAR Hanoi Open'], axis=1, inplace=True)
df_3.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '휴온스 LPBA 챔피언십'], axis=1, inplace=True)

df = pd.concat([df_1, df_2, df_3], join='outer', axis=0, ignore_index=True)
df['image_name'] = co['image_name']



coordinate_columns = ['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']
co_clean = co.dropna(subset=coordinate_columns)

merged_df = pd.merge(co_clean, df, on='image_name')



# 첫 번째 경로와 두 번째 경로 지정
first_dir = "/content/drive/MyDrive/Colab Notebooks/LPBA/Created"
second_dir = "/content/drive/MyDrive/Colab Notebooks/LPBA_2/Created"
third_dir = "/content/drive/MyDrive/Colab Notebooks/LPBA_3/Created"

# 이미지 파일 불러오기
first_images = sorted(glob.glob(os.path.join(first_dir, "*.png")))
second_images = sorted(glob.glob(os.path.join(second_dir, "*.png")))
third_images = sorted(glob.glob(os.path.join(third_dir, "*.png")))

# 결과를 저장할 리스트
image_paths = []

# 경로의 이미지를 그대로 추가
image_paths.extend(first_images)
image_paths.extend(second_images)
image_paths.extend(third_images)



# 이미지 로드 및 전처리 함수 정의
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = preprocess_input(image)

    return image



# 이미지와 타겟 데이터를 배열로 저장
images = []
labels = []

for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0]):
    image_name = str(int(row['image_name'])).zfill(4) + ".png"  # 이미지 이름을 4자리로 맞춤

    # 이미지 경로를 직접 생성 (이름에 맞는 경로를 찾음)
    if int(image_name[:-4]) <= 2163:
        # 첫 번째 디렉토리의 이미지
        image_path = os.path.join(first_dir, image_name)

    elif int(image_name[:-4]) <= 4584:
        # 두 번째 디렉토리의 이미지
        image_path = os.path.join(second_dir, image_name)

    else:
        # 세 번째 디렉토리의 이미지
        image_path = os.path.join(third_dir, image_name)

    if not os.path.exists(image_path):
        print(f"Error: Image {image_name} not found in image paths.")

        continue

    label = row['Results']  # df의 타겟 열 이름으로 수정

    # 이미지 불러오기 및 전처리
    try:
        image = load_and_preprocess_image(image_path)
        images.append(image)
        labels.append(label)

    except Exception as e:
        print(f"Error: Failed at loading image {image_name}: {e}")



# numpy 배열로 변환
images = np.array(images)
labels = to_categorical(labels)

print("image shape:", images.shape)
print("label shape:", labels.shape)



# 이미지 이름 목록을 가져옵니다 (이미지 이름 리스트가 있어야 함)
image_names = merged_df['image_name'].tolist()



print(len(image_names))
print(len(images))
print(len(labels))

100%|██████████| 6268/6268 [54:14<00:00,  1.93it/s]


image shape: (6268, 224, 224, 3)
label shape: (6268, 2)
6268
6268
6268


In [ ]:
# 데이터셋을 학습용/검증용으로 분할
X_train, X_val, y_train, y_val, train_image_names, val_image_names = train_test_split(
    images, labels, image_names, test_size=0.3, random_state=42, shuffle=True
)

# 배치 처리: tf.data를 이용한 효율적인 배치 학습
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)



# ResNet50 모델 로드 (ImageNet 가중치 사용, 최상위 층은 제거)
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False  # 기존 ResNet50 가중치는 학습되지 않도록 설정 (전이 학습)

# ResNet50 기반의 CNN 모델 구성
model = Sequential()
model.add(resnet50)  # ResNet50 모델 추가
model.add(Flatten())  # 펼치기

# 배치 정규화 및 드롭아웃 추가
model.add(Dense(128, activation=None, kernel_regularizer=l2(0.02), kernel_initializer=HeNormal()))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

# 출력층 (분류 문제에 맞게 조정)
model.add(Dense(2, activation='softmax'))  # 출력층

# Fine-tuning을 위한 층 동결 해제
for layer in resnet50.layers[-10:]:
    layer.trainable = True

# 모델 컴파일 (Adam 옵티마이저 사용)
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)

optimizer = Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True,
                               min_delta = 0.001,
                               verbose = 1)



# 모델 학습
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=[early_stopping])



# 모델 성능 평가
test_loss, test_accuracy = model.evaluate(val_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 40s 156ms/step - accuracy: 0.5345 - loss: 3.4714 - val_accuracy: 0.5460 - val_loss: 1.4317
Epoch 2/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.5385 - loss: 1.2850 - val_accuracy: 0.5582 - val_loss: 1.0520
Epoch 3/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.5051 - loss: 1.0343 - val_accuracy: 0.5572 - val_loss: 0.9360
Epoch 4/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.5474 - loss: 0.9055 - val_accuracy: 0.5635 - val_loss: 0.8428
Epoch 5/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.5414 - loss: 0.8831 - val_accuracy: 0.5630 - val_loss: 0.8812
Epoch 6/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.5378 - loss: 0.9204 - val_accuracy: 0.5561 - val_loss: 0.9827
Epoch 7/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.5405 - loss: 0.9773 - val_accuracy: 0.5444 - val_loss: 0.9273
Epoch 8/50
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 6

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/LPBA_3/TheNew_ResNet50_v1.keras')

In [ ]:
# 모델 예측 확률 계산
train_pred_proba = model.predict(X_train)  # (num_train_samples, 1)
val_pred_proba = model.predict(X_val)     # (num_val_samples, 1)



train_results_df = pd.DataFrame({'image_name': train_image_names, 'pred_proba': train_pred_proba[:, 1]})
train_results_df['image_name'] = train_results_df['image_name'].astype(str).str.zfill(4)

val_results_df = pd.DataFrame({'image_name': val_image_names, 'pred_proba': val_pred_proba[:, 1]})
val_results_df['image_name'] = val_results_df['image_name'].astype(str).str.zfill(4)



# Visual Python: Data Analysis > Bind
merged_results_df = pd.concat([train_results_df,val_results_df], join='outer', axis=0, ignore_index=True)

# image_name을 기준으로 오름차순 정렬
merged_results_df = merged_results_df.sort_values(by='image_name', ascending=True).reset_index(drop=True)

merged_df['pred_proba'] = merged_results_df['pred_proba']



train_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/TheNew_ResNet50_v1_proba_train.csv')
val_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/TheNew_ResNet50_v1_proba_val.csv')
merged_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/TheNew_ResNet50_v1_proba_merged.csv')

138/138 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step


## ResNet 2

In [ ]:
co_1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA/Coordinate.csv')
co_2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_2/Coordinate.csv')
co_3 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

co_1.drop('Unnamed: 0', axis=1, inplace=True)
co_2.drop('Unnamed: 0', axis=1, inplace=True)
co_3.drop('Unnamed: 0', axis=1, inplace=True)

co = pd.concat([co_1,co_2, co_3], join='outer', axis=0, ignore_index=True)
co = co - 13

co.reset_index(inplace=True)
co.rename(columns = {'index': 'image_name'}, inplace = True)

co['image_name'] = co['image_name'] + 1



df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA/Data.xlsx')
df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_2/Data.xlsx')
df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')

df_1.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '크라운해태 LPBA 챔피언십 2024 한가위'], axis=1, inplace=True)
df_2.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '2024 LPBA SYBAZZAR Hanoi Open'], axis=1, inplace=True)
df_3.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '휴온스 LPBA 챔피언십'], axis=1, inplace=True)

df = pd.concat([df_1, df_2, df_3], join='outer', axis=0, ignore_index=True)
df['image_name'] = co['image_name']



coordinate_columns = ['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']
co_clean = co.dropna(subset=coordinate_columns)

merged_df = pd.merge(co_clean, df, on='image_name')

In [ ]:
# save_path = '/content/drive/MyDrive/Colab Notebooks/LPBA_3'

# # numpy 배열로 저장
# np.save(f"{save_path}/images.npy", images)  # images 저장
# np.save(f"{save_path}/labels.npy", labels)  # labels 저장

# # image_names 저장 (pickle 사용)
# with open(f"{save_path}/image_names.pkl", "wb") as f:
#     pickle.dump(image_names, f)

# print("Data saved successfully.")

Data saved successfully.


In [ ]:
load_path = '/content/drive/MyDrive/Colab Notebooks/LPBA_3'

# numpy 배열로 불러오기
images = np.load(f"{load_path}/images.npy")  # images 로드
labels = np.load(f"{load_path}/labels.npy")  # labels 로드

# image_names 로드 (pickle 사용)
with open(f"{load_path}/image_names.pkl", "rb") as f:
    image_names = pickle.load(f)

print("Data loaded successfully.")
print("Image shape:", images.shape)
print("Label shape:", labels.shape)
print("Number of image names:", len(image_names))

Data loaded successfully.
Image shape: (6268, 224, 224, 3)
Label shape: (6268, 2)
Number of image names: 6268


In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Pre-trained ResNet50 모델 로드 (ImageNet 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 커스터마이징
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.6)(x)
predictions = Dense(2, activation='softmax', kernel_regularizer=l2(0.01))(x)  # 2개의 클래스 (0과 1)

model = Model(inputs=base_model.input, outputs=predictions)

# Base model의 가중치를 동결
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=1
)

# Base model의 가중치 동결 해제 (fine-tuning)
for layer in base_model.layers:
    layer.trainable = True

# 모델 재컴파일
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# 학습률 감소 콜백 설정
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    factor=0.5,
    min_lr=1e-6,
    verbose=1
)

# 모델 재학습
history_finetune = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[lr_reduction],
    verbose=1
)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_acc}")

# 분류 보고서
y_pred = model.predict(X_test)
y_pred_classes = tf.argmax(y_pred, axis=1)
y_true = tf.argmax(y_test, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

# Confusion Matrix 시각화
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 8))
plt.imshow(conf_matrix, cmap='Blues')
plt.title("Confusion Matrix")
plt.colorbar()
plt.show()

# 학습 곡선 시각화
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history_finetune.history['accuracy'], label='Fine-tune Train Accuracy')
plt.plot(history_finetune.history['val_accuracy'], label='Fine-tune Validation Accuracy')
plt.legend()
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

## ResNet 3

In [4]:
co_1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA/Coordinate.csv')
co_2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_2/Coordinate.csv')
co_3 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

co_1.drop('Unnamed: 0', axis=1, inplace=True)
co_2.drop('Unnamed: 0', axis=1, inplace=True)
co_3.drop('Unnamed: 0', axis=1, inplace=True)

co = pd.concat([co_1,co_2, co_3], join='outer', axis=0, ignore_index=True)
co = co - 13

co.reset_index(inplace=True)
co.rename(columns = {'index': 'image_name'}, inplace = True)

co['image_name'] = co['image_name'] + 1



df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA/Data.xlsx')
df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_2/Data.xlsx')
df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')

df_1.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '크라운해태 LPBA 챔피언십 2024 한가위'], axis=1, inplace=True)
df_2.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '2024 LPBA SYBAZZAR Hanoi Open'], axis=1, inplace=True)
df_3.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '휴온스 LPBA 챔피언십'], axis=1, inplace=True)

df = pd.concat([df_1, df_2, df_3], join='outer', axis=0, ignore_index=True)
df['image_name'] = co['image_name']



coordinate_columns = ['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']
co_clean = co.dropna(subset=coordinate_columns)

merged_df = pd.merge(co_clean, df, on='image_name')



load_path = '/content/drive/MyDrive/Colab Notebooks/LPBA_3'

# numpy 배열로 불러오기
images = np.load(f"{load_path}/images.npy")  # images 로드
labels = np.load(f"{load_path}/labels.npy")  # labels 로드

# image_names 로드 (pickle 사용)
with open(f"{load_path}/image_names.pkl", "rb") as f:
    image_names = pickle.load(f)

print("Data loaded successfully.")
print("Image shape:", images.shape)
print("Label shape:", labels.shape)
print("Number of image names:", len(image_names))

Data loaded successfully.
Image shape: (6268, 224, 224, 3)
Label shape: (6268, 2)
Number of image names: 6268


In [7]:
merged_df

,image_name,큐볼_x,큐볼_y,적구_1_x,적구_1_y,적구_2_x,적구_2_y,Results
0,1,565.0,186.0,378.0,184.0,191.0,137.0,0
1,2,357.0,46.0,84.0,277.0,197.0,320.0,0
2,3,533.0,356.0,553.0,200.0,689.0,242.0,0
3,4,21.0,21.0,563.0,112.0,711.0,131.0,0
4,5,496.0,15.0,704.0,298.0,712.0,128.0,1
...,...,...,...,...,...,...,...,...
6263,6695,220.0,250.0,22.0,308.0,110.0,345.0,1
6264,6696,116.0,345.0,33.0,219.0,151.0,349.0,0
6265,6697,617.0,345.0,454.0,54.0,256.0,279.0,1
6266,6698,469.0,337.0,456.0,81.0,446.0,236.0,0


In [8]:
# # 1. 좌표 데이터 전처리
# coords = merged_df[['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']].values
# scaler = MinMaxScaler()
# coords_scaled = scaler.fit_transform(coords)

# # 데이터 분할
# X_train_img, X_test_img, X_train_coords, X_test_coords, y_train, y_test = train_test_split(
#     images, coords_scaled, labels, test_size=0.2, random_state=42
# )

# # 2. ResNet50 (이미지 처리 분기)
# image_input = Input(shape=(224, 224, 3))
# base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(512, activation=None, kernel_regularizer=l2(0.01))(x)  # Activation 없애고 BatchNorm 추가
# x = BatchNormalization()(x)
# x = tf.keras.layers.Activation('relu')(x)
# x = Dropout(0.5)(x)
# image_branch = Dense(256, activation=None)(x)
# image_branch = BatchNormalization()(image_branch)
# image_branch = tf.keras.layers.Activation('relu')(image_branch)

# # 3. 좌표 처리 분기
# coords_input = Input(shape=(6,))
# y = Dense(64, activation=None, kernel_regularizer=l2(0.01))(coords_input)  # Activation 없애고 BatchNorm 추가
# y = BatchNormalization()(y)
# y = tf.keras.layers.Activation('relu')(y)
# y = Dropout(0.5)(y)
# coords_branch = Dense(32, activation=None)(y)
# coords_branch = BatchNormalization()(coords_branch)
# coords_branch = tf.keras.layers.Activation('relu')(coords_branch)

# # 4. 두 분기 결합
# merged = Concatenate()([image_branch, coords_branch])
# z = Dense(128, activation=None, kernel_regularizer=l2(0.01))(merged)  # Activation 없애고 BatchNorm 추가
# z = BatchNormalization()(z)
# z = tf.keras.layers.Activation('relu')(z)
# z = Dropout(0.5)(z)
# output = Dense(2, activation='softmax')(z)

# # 5. 모델 정의
# model = Model(inputs=[image_input, coords_input], outputs=output)

# # 6. 모델 컴파일
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# # 7. 콜백 설정
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
# lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6, verbose=1)

# # 8. 모델 학습
# history = model.fit(
#     [X_train_img, X_train_coords], y_train,
#     validation_split=0.2,
#     epochs=50,
#     batch_size=32,
#     callbacks=[early_stopping, lr_reduction],
#     verbose=1
# )

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 118s 410ms/step - accuracy: 0.4951 - loss: 10.9396 - val_accuracy: 0.4985 - val_loss: 10.1422 - learning_rate: 1.0000e-04
Epoch 2/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5243 - loss: 10.1953 - val_accuracy: 0.5374 - val_loss: 9.4690 - learning_rate: 1.0000e-04
Epoch 3/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5106 - loss: 9.4982 - val_accuracy: 0.5244 - val_loss: 8.7962 - learning_rate: 1.0000e-04
Epoch 4/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5151 - loss: 8.7959 - val_accuracy: 0.5254 - val_loss: 8.1233 - learning_rate: 1.0000e-04
Epoch 5/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5243 - loss: 8.0990 - val_accuracy: 0.5284 - val_loss: 7.4734 - learning_rate: 1.0000e-04
Epoch 6/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5060 - loss: 7.4458 - val_accuracy: 0.5294 - val_loss: 6.8322 - learning_rate: 1.0000e-04
E

KeyboardInterrupt: 

In [ ]:
# 9. 모델 평가
test_loss, test_acc = model.evaluate([X_test_img, X_test_coords], y_test, verbose=1)
print(f"Test Accuracy: {test_acc}")

# 10. 시각화 (학습 곡선)
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

## ResNet 4

In [6]:
co_1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA/Coordinate.csv')
co_2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_2/Coordinate.csv')
co_3 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

co_1.drop('Unnamed: 0', axis=1, inplace=True)
co_2.drop('Unnamed: 0', axis=1, inplace=True)
co_3.drop('Unnamed: 0', axis=1, inplace=True)

co = pd.concat([co_1,co_2, co_3], join='outer', axis=0, ignore_index=True)
co = co - 13

co.reset_index(inplace=True)
co.rename(columns = {'index': 'image_name'}, inplace = True)

co['image_name'] = co['image_name'] + 1



df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA/Data.xlsx')
df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_2/Data.xlsx')
df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')

df_1.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '크라운해태 LPBA 챔피언십 2024 한가위'], axis=1, inplace=True)
df_2.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '2024 LPBA SYBAZZAR Hanoi Open'], axis=1, inplace=True)
df_3.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '휴온스 LPBA 챔피언십'], axis=1, inplace=True)

df = pd.concat([df_1, df_2, df_3], join='outer', axis=0, ignore_index=True)
df['image_name'] = co['image_name']



coordinate_columns = ['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']
co_clean = co.dropna(subset=coordinate_columns)

merged_df = pd.merge(co_clean, df, on='image_name')



load_path = '/content/drive/MyDrive/Colab Notebooks/LPBA_3'

# numpy 배열로 불러오기
images = np.load(f"{load_path}/images.npy")  # images 로드
labels = np.load(f"{load_path}/labels.npy")  # labels 로드

# image_names 로드 (pickle 사용)
with open(f"{load_path}/image_names.pkl", "rb") as f:
    image_names = pickle.load(f)

print("Data loaded successfully.")
print("Image shape:", images.shape)
print("Label shape:", labels.shape)
print("Number of image names:", len(image_names))

Data loaded successfully.
Image shape: (6268, 224, 224, 3)
Label shape: (6268, 2)
Number of image names: 6268


In [5]:
# 데이터셋을 학습용/검증용으로 분할
X_train, X_val, y_train, y_val, train_image_names, val_image_names = train_test_split(
    images, labels, image_names, test_size=0.2, random_state=42, shuffle=True
)

# 배치 처리: tf.data를 이용한 효율적인 배치 학습
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)


# 데이터 증강 레이어 정의
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),  # 최대 10% 회전
    tf.keras.layers.RandomFlip('horizontal')  # 좌우 뒤집기
])

# 데이터 증강 적용된 데이터셋 구성
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .map(lambda x, y: (data_augmentation(x), y)) \
    .shuffle(buffer_size=1000) \
    .batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

# ResNet50 모델 로드 (ImageNet 가중치 사용, 최상위 층은 제거)
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False  # 기존 ResNet50 가중치는 학습되지 않도록 설정 (전이 학습)

# ResNet50 기반의 CNN 모델 구성
model = Sequential()
model.add(resnet50)  # ResNet50 모델 추가
model.add(Flatten())  # 펼치기

# 배치 정규화 및 드롭아웃 추가
model.add(Dense(128, activation=None, kernel_regularizer=l2(0.01), kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.6))

# 출력층 (분류 문제에 맞게 조정)
model.add(Dense(2, activation='softmax'))  # 출력층

# Fine-tuning을 위한 층 동결 해제
for layer in resnet50.layers[-4:]:
    layer.trainable = True

# 모델 컴파일 (Adam 옵티마이저 사용)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True,
                               min_delta=0.001,
                               verbose=1)

# 모델 학습
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=[early_stopping])

# 모델 성능 평가
test_loss, test_accuracy = model.evaluate(val_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 64s 274ms/step - accuracy: 0.5045 - loss: 3.3151 - val_accuracy: 0.5327 - val_loss: 2.4034
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 197ms/step - accuracy: 0.5115 - loss: 2.4508 - val_accuracy: 0.5423 - val_loss: 1.9067
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 198ms/step - accuracy: 0.5244 - loss: 2.0126 - val_accuracy: 0.5351 - val_loss: 1.5910
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 197ms/step - accuracy: 0.5361 - loss: 1.7304 - val_accuracy: 0.5191 - val_loss: 1.4201
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 197ms/step - accuracy: 0.5279 - loss: 1.5330 - val_accuracy: 0.5566 - val_loss: 1.2461
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 198ms/step - accuracy: 0.5389 - loss: 1.3723 - val_accuracy: 0.5183 - val_loss: 1.1753
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 197ms/step - accuracy: 0.5368 - loss: 1.2742 - val_accuracy: 0.5470 - val_loss: 1.0727
Epoch 8/50
157/157 ━━━━━━━━━━━━━

KeyboardInterrupt: 

# CNN

## Preparing

In [ ]:
co_1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA/Coordinate.csv')
co_2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_2/Coordinate.csv')
co_3 = pd.read_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/Coordinate.csv')

co_1.drop('Unnamed: 0', axis=1, inplace=True)
co_2.drop('Unnamed: 0', axis=1, inplace=True)
co_3.drop('Unnamed: 0', axis=1, inplace=True)

co = pd.concat([co_1,co_2, co_3], join='outer', axis=0, ignore_index=True)
co = co - 13

co.reset_index(inplace=True)
co.rename(columns = {'index': 'image_name'}, inplace = True)

co['image_name'] = co['image_name'] + 1

In [ ]:
df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA/Data.xlsx')
df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_2/Data.xlsx')
df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/LPBA_3/Data.xlsx')

df_1.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '크라운해태 LPBA 챔피언십 2024 한가위'], axis=1, inplace=True)
df_2.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '2024 LPBA SYBAZZAR Hanoi Open'], axis=1, inplace=True)
df_3.drop(['Score', 'Ball', 'Player', 'Compet', 'Match', 'Set', '휴온스 LPBA 챔피언십'], axis=1, inplace=True)

df = pd.concat([df_1, df_2, df_3], join='outer', axis=0, ignore_index=True)
df['image_name'] = co['image_name']



coordinate_columns = ['큐볼_x', '큐볼_y', '적구_1_x', '적구_1_y', '적구_2_x', '적구_2_y']
co_clean = co.dropna(subset=coordinate_columns)

merged_df = pd.merge(co_clean, df, on='image_name')

In [ ]:
# 첫 번째 경로와 두 번째 경로 지정
first_dir = "/content/drive/MyDrive/Colab Notebooks/LPBA/Created"
second_dir = "/content/drive/MyDrive/Colab Notebooks/LPBA_2/Created"
third_dir = "/content/drive/MyDrive/Colab Notebooks/LPBA_3/Created"

# 이미지 파일 불러오기
first_images = sorted(glob.glob(os.path.join(first_dir, "*.png")))
second_images = sorted(glob.glob(os.path.join(second_dir, "*.png")))
third_images = sorted(glob.glob(os.path.join(third_dir, "*.png")))

# 결과를 저장할 리스트
image_paths = []

# 경로의 이미지를 그대로 추가
image_paths.extend(first_images)
image_paths.extend(second_images)
image_paths.extend(third_images)



# 이미지 로드 및 전처리 함수 정의
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image)
    image = preprocess_input(image)

    return image



# 이미지와 타겟 데이터를 배열로 저장
images = []
labels = []

for index, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0]):
    image_name = str(int(row['image_name'])).zfill(4) + ".png"  # 이미지 이름을 4자리로 맞춤

    # 이미지 경로를 직접 생성 (이름에 맞는 경로를 찾음)
    if int(image_name[:-4]) <= 2163:
        # 첫 번째 디렉토리의 이미지
        image_path = os.path.join(first_dir, image_name)

    elif int(image_name[:-4]) <= 4584:
        # 두 번째 디렉토리의 이미지
        image_path = os.path.join(second_dir, image_name)

    else:
        # 세 번째 디렉토리의 이미지
        image_path = os.path.join(third_dir, image_name)

    if not os.path.exists(image_path):
        print(f"Error: Image {image_name} not found in image paths.")

        continue

    label = row['Results']  # df의 타겟 열 이름으로 수정

    # 이미지 불러오기 및 전처리
    try:
        image = load_and_preprocess_image(image_path)
        images.append(image)
        labels.append(label)

    except Exception as e:
        print(f"Error: Failed at loading image {image_name}: {e}")

100%|██████████| 6268/6268 [14:11<00:00,  7.36it/s]


## 모든 대회 이미지 학습

In [ ]:
# numpy 배열로 변환
images = np.array(images)
labels = to_categorical(labels)

print("image shape:", images.shape)
print("label shape:", labels.shape)



# 이미지 이름 목록을 가져옵니다 (이미지 이름 리스트가 있어야 함)
image_names = merged_df['image_name'].tolist()



print(len(image_names))
print(len(images))
print(len(labels))

image shape: (6268, 224, 224, 3)
label shape: (6268, 2)
6268
6268
6268


In [ ]:
# 배치 처리: tf.data를 이용한 효율적인 배치 학습
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels)).shuffle(buffer_size=1000).batch(batch_size)  # images, labels 사용

In [ ]:
print(len(image_names))
print(len(images))
print(len(labels))

### ResNet

In [ ]:
# ResNet50 모델 로드 (ImageNet 가중치 사용, 최상위 층은 제거)
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False  # 기존 ResNet50 가중치는 학습되지 않도록 설정 (전이 학습)

# ResNet50 기반의 CNN 모델 구성
model = Sequential()
model.add(resnet50)  # ResNet50 모델 추가
model.add(Flatten())  # 펼치기

# 배치 정규화 및 드롭아웃 추가
model.add(Dense(128, activation=None, kernel_regularizer=l2(0.01), kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.6))

# 출력층 (분류 문제에 맞게 조정)
model.add(Dense(2, activation='softmax'))  # 출력층

# Fine-tuning을 위한 층 동결 해제
for layer in resnet50.layers[-4:]:
    layer.trainable = True

# 모델 컴파일 (Adam 옵티마이저 사용)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# EarlyStopping 콜백 정의 (검증 데이터가 없으므로 'loss'를 모니터링)
early_stopping = EarlyStopping(
    monitor='loss',               # 'val_loss' -> 'loss'로 변경
    patience=5,
    restore_best_weights=True,
    min_delta=0.001,
    verbose=1
)

# 모델 학습 (validation_data 제거)
history = model.fit(
    train_dataset,
    epochs=50,
    callbacks=[early_stopping],   # 검증 데이터가 없으므로 early_stopping은 'loss' 모니터링
    verbose=1
)

# 모델 성능 평가 (train_dataset으로 평가)
train_loss, train_accuracy = model.evaluate(train_dataset)
print(f"Train Loss: {train_loss}")
print(f"Train Accuracy: {train_accuracy}")

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/LPBA_3/model_ResNet50_v0.keras')

In [ ]:
# 모델 예측 확률 계산 (전체 데이터를 훈련용으로 사용)
train_pred_proba = model.predict(images)  # 전체 이미지에 대해 예측

# 예측 결과를 데이터프레임으로 변환
train_results_df = pd.DataFrame({'image_name': image_names, 'pred_proba': train_pred_proba[:, 1]})
train_results_df['image_name'] = train_results_df['image_name'].astype(str).str.zfill(4)

# CSV 파일로 저장
train_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/train_v0.csv', index=False)

196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step


## 모든 대회 이미지 학습 및 테스트

In [ ]:
# numpy 배열로 변환
images = np.array(images)
labels = to_categorical(labels)

print("image shape:", images.shape)
print("label shape:", labels.shape)



# 이미지 이름 목록을 가져옵니다 (이미지 이름 리스트가 있어야 함)
image_names = merged_df['image_name'].tolist()



print(len(image_names))
print(len(images))
print(len(labels))

image shape: (6268, 224, 224, 3)
label shape: (6268, 2)
6268
6268
6268


In [7]:
# 데이터셋을 학습용/검증용으로 분할
X_train, X_val, y_train, y_val, train_image_names, val_image_names = train_test_split(
    images, labels, image_names, test_size=0.2, random_state=42, shuffle=True
)

# 배치 처리: tf.data를 이용한 효율적인 배치 학습
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

### ResNet

In [8]:
# ResNet50 모델 로드 (ImageNet 가중치 사용, 최상위 층은 제거)
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False  # 기존 ResNet50 가중치는 학습되지 않도록 설정 (전이 학습)

# ResNet50 기반의 CNN 모델 구성
model = Sequential()
model.add(resnet50)  # ResNet50 모델 추가
model.add(Flatten())  # 펼치기

# 배치 정규화 및 드롭아웃 추가
model.add(Dense(128, activation=None, kernel_regularizer=l2(0.01), kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.6))

# 출력층 (분류 문제에 맞게 조정)
model.add(Dense(2, activation='softmax'))  # 출력층

# Fine-tuning을 위한 층 동결 해제
for layer in resnet50.layers[-4:]:
    layer.trainable = True

# 모델 컴파일 (Adam 옵티마이저 사용)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True,
                               min_delta = 0.001,
                               verbose = 1)

# 모델 학습
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=[early_stopping])



# 모델 성능 평가
test_loss, test_accuracy = model.evaluate(val_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 29s 107ms/step - accuracy: 0.5163 - loss: 3.2394 - val_accuracy: 0.5646 - val_loss: 2.2398
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.5879 - loss: 2.1562 - val_accuracy: 0.5686 - val_loss: 1.7268
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.5896 - loss: 1.7332 - val_accuracy: 0.5518 - val_loss: 1.5072
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.6162 - loss: 1.4805 - val_accuracy: 0.5917 - val_loss: 1.3121
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.6422 - loss: 1.2692 - val_accuracy: 0.5494 - val_loss: 1.2448
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.6591 - loss: 1.1713 - val_accuracy: 0.5718 - val_loss: 1.1648
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.6726 - loss: 1.0742 - val_accuracy: 0.5781 - val_loss: 1.1529
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.7061 - loss: 0.9793 - val_ac

In [9]:
model.save('/content/drive/MyDrive/Colab Notebooks/LPBA_3/model_ResNet50_v1.keras')

In [10]:
# 모델 예측 확률 계산
train_pred_proba = model.predict(X_train)  # (num_train_samples, 1)
val_pred_proba = model.predict(X_val)     # (num_val_samples, 1)



train_results_df = pd.DataFrame({'image_name': train_image_names, 'pred_proba': train_pred_proba[:, 1]})
train_results_df['image_name'] = train_results_df['image_name'].astype(str).str.zfill(4)

val_results_df = pd.DataFrame({'image_name': val_image_names, 'pred_proba': val_pred_proba[:, 1]})
val_results_df['image_name'] = val_results_df['image_name'].astype(str).str.zfill(4)



# Visual Python: Data Analysis > Bind
merged_results_df = pd.concat([train_results_df,val_results_df], join='outer', axis=0, ignore_index=True)

# image_name을 기준으로 오름차순 정렬
merged_results_df = merged_results_df.sort_values(by='image_name', ascending=True).reset_index(drop=True)

merged_df['pred_proba'] = merged_results_df['pred_proba']



train_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/train_v1.csv')
val_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/val_v1.csv')
merged_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/merged_v1.csv')

157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step


## 1번 대회 제외 이미지 학습 및 1번 대회 이미지로 테스트

In [ ]:
# 1. 이미지 번호를 정수형으로 변환
image_names = [int(name) for name in image_names]

# 2. 학습용 및 검증용 데이터 분할
val_indices = [i for i, name in enumerate(image_names) if name <= 2163]
train_indices = [i for i, name in enumerate(image_names) if name > 2163]



# 3. 학습용 데이터셋
X_train = images[train_indices]
y_train = labels[train_indices]
train_image_names = [str(image_names[i]).zfill(4) for i in train_indices]

# 4. 검증용 데이터셋
X_val = images[val_indices]
y_val = labels[val_indices]
val_image_names = [str(image_names[i]).zfill(4) for i in val_indices]



# 5. 배치 처리: tf.data를 이용한 효율적인 배치 학습
batch_size = 32

# 학습용 데이터셋
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(batch_size)

# 검증용 데이터셋
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

# 데이터셋 확인
print(f"학습 데이터 개수: {len(X_train)}")
print(f"검증 데이터 개수: {len(X_val)}")

### ResNet

In [ ]:
# ResNet50 모델 로드 (ImageNet 가중치 사용, 최상위 층은 제거)
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50.trainable = False  # 기존 ResNet50 가중치는 학습되지 않도록 설정 (전이 학습)

# ResNet50 기반의 CNN 모델 구성
model = Sequential()
model.add(resnet50)  # ResNet50 모델 추가
model.add(Flatten())  # 펼치기

# 배치 정규화 및 드롭아웃 추가
model.add(Dense(128, activation=None, kernel_regularizer=l2(0.01), kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.6))

# 출력층 (분류 문제에 맞게 조정)
model.add(Dense(2, activation='softmax'))  # 출력층

# Fine-tuning을 위한 층 동결 해제
for layer in resnet50.layers[-4:]:
    layer.trainable = True

# 모델 컴파일 (Adam 옵티마이저 사용)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True,
                               min_delta = 0.001,
                               verbose = 1)

# 모델 학습
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=[early_stopping])



# 모델 성능 평가
test_loss, test_accuracy = model.evaluate(val_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/LPBA_3/model_ResNet50_v2.keras')

In [ ]:
# 모델 예측 확률 계산
train_pred_proba = model.predict(X_train)  # (num_train_samples, 1)
val_pred_proba = model.predict(X_val)     # (num_val_samples, 1)



train_results_df = pd.DataFrame({'image_name': train_image_names, 'pred_proba': train_pred_proba[:, 1]})
train_results_df['image_name'] = train_results_df['image_name'].astype(str).str.zfill(4)

val_results_df = pd.DataFrame({'image_name': val_image_names, 'pred_proba': val_pred_proba[:, 1]})
val_results_df['image_name'] = val_results_df['image_name'].astype(str).str.zfill(4)



# Visual Python: Data Analysis > Bind
merged_results_df = pd.concat([train_results_df,val_results_df], join='outer', axis=0, ignore_index=True)

# image_name을 기준으로 오름차순 정렬
merged_results_df = merged_results_df.sort_values(by='image_name', ascending=True).reset_index(drop=True)

merged_df['pred_proba'] = merged_results_df['pred_proba']



train_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/train_v2.csv')
val_results_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/val_v2.csv')
merged_df.to_csv('./drive/MyDrive/Colab Notebooks/LPBA_3/merged_v2.csv')